In [1]:
'''import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials
 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id = '1-lri-HK_aq_UhHnprQTi5zg5sqldKVhm'  #雲端硬碟檔案連結碼
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('yahoo_total_data.csv')
data = pd.read_csv('yahoo_total_data.csv')
print(data)'''

"import pandas as pd\nfrom pydrive.auth import GoogleAuth\nfrom pydrive.drive import GoogleDrive \nfrom google.colab import auth \nfrom oauth2client.client import GoogleCredentials\n \nauth.authenticate_user()\ngauth = GoogleAuth()\ngauth.credentials = GoogleCredentials.get_application_default()\ndrive = GoogleDrive(gauth)\nfile_id = '1-lri-HK_aq_UhHnprQTi5zg5sqldKVhm'  #雲端硬碟檔案連結碼\ndownloaded = drive.CreateFile({'id': file_id})\ndownloaded.GetContentFile('yahoo_total_data.csv')\ndata = pd.read_csv('yahoo_total_data.csv')\nprint(data)"

In [2]:
#!pip install scikit-optimize

In [3]:
import os
import time
import pandas as pd
import numpy as np
import time
from datetime import datetime
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from skopt import gp_minimize #貝氏優化
from skopt.space import Real, Integer
from skopt.utils import use_named_args
print(tf.__version__)

2.0.0


In [4]:
df = pd.read_csv('/PAPER/SRCNN/yahoo/yahoo_total_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572966 entries, 0 to 572965
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  572966 non-null  int64  
 1   timestamp   572966 non-null  int64  
 2   value       572966 non-null  float64
 3   is_anomaly  572966 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 17.5 MB


## Generate train data (有Label)

In [5]:
class gen():
    def __init__(self, win_siz, step, nums):
        self.control = 0
        self.win_siz = win_siz
        self.step = step
        self.number = nums

    def generate_train_data(self, value, label, train_op, back_k=0):
        def normalize(a):
            amin = np.min(a)
            amax = np.max(a)
            a = (a - amin) / (amax - amin + 1e-5)
            return 3 * a

        if back_k <= 5:
            back = back_k
        else:
            back = 5
        length = len(value)
        tmp = []
        for pt in range(self.win_siz, length - back, self.step): # window=128 step=64 跳一次
            
            head = max(0, pt - self.win_siz)
            tail = min(length - back, pt)
            data = normalize(np.array(value[head:tail]).astype(np.float64)) # 以 window=128 做切割 (128,1)
            #lbs = np.array(label[head:tail]).astype(np.float64)    #有 label
            lbs = np.zeros(self.win_siz, dtype=np.int64)

            num = np.random.randint(1, self.number)   # [1, number] 之間隨機選一個整數 num
            ids = np.random.choice(self.win_siz, num, replace=False)  # 再從 window 之中選 num 個數 為異常
            
            if train_op:
                if (self.win_siz - 6) not in ids:    # control 累積到 100 若沒有 (window - 6) 在選擇內 則加進去 
                    self.control += np.random.random()  # [0,1] 之間隨機數
                else:
                    self.control = 0
                if self.control > 100:
                    ids[0] = self.win_siz - 6
                    self.control = 0

                mean = np.mean(data)
                dataavg = average_filter(data)
                var = np.var(data)

                for id in ids: # 將原始data加上異常以覆蓋  np.random.standard_cauchy / np.random.randn() / np.random.standard_t(10)
                    data[id] += (dataavg[id] + mean) * np.random.randn() * min((1 + var), 10)   # 製造異常 (保有Label 異常為1)
                    lbs[id] = 1
            
            tmp.append([data.tolist(), lbs.tolist()])
            
        return tmp

In [6]:
def average_filter(values, n=3):
    """
    Calculate the sliding window average for the give time series.
    Mathematically, res[i] = sum_{j=i-t+1}^{i} values[j] / t, where t = min(n, i+1)
    :param values: list.
        a list of float numbers
    :param n: int, default 3.
        window size.
    :return res: list.
        a list of value after the average_filter process.
    """

    if n >= len(values):
        n = len(values)

    res = np.cumsum(values, dtype=float)
    res[n:] = res[n:] - res[:-n]
    res[n:] = res[n:] / n

    for i in range(1, n):
        res[i] /= (i + 1)

    return res

## Not One class data (自己加上異常並標全新Label)

In [7]:
window = 64
step = 32
num = 5  # upper limit value for the number of anomaly points

if __name__ == '__main__':

    results = []
    print("generating train data")
    generator = gen(window, step, num)
    
    in_timestamp, in_value, in_label = df['timestamp'], df['value'], df['is_anomaly']

    if len(in_value) < window:
        print("value's length < window size", len(in_value), window)
    
    half_len = int(len(in_value)/2)
    gen_data = generator.generate_train_data(in_value[:half_len], in_label[:half_len], True)  # [in_label==0] 丟原始異常
    X_train = np.array(gen_data).astype('float32')
    
    print('total fake data size:', len(gen_data))
    print('gen_data shape', np.array(gen_data).shape)
    print('train shape:', np.array(X_train).shape)
    

generating train data
total fake data size: 8951
gen_data shape (8951, 2, 64)
train shape: (8951, 2, 64)


In [8]:
print(sum(in_label==1))
print('異常點比例：', sum(in_label==1)/in_value.shape[0])

3915
異常點比例： 0.006832866173560037


## 未轉SR的Time Series與Label標記情況(前100筆window)

In [9]:
'''for i in range(100):
    print('picture  ', i)
    plt.plot(X_train[:,0][i])
    index_changes = np.where(X_train[:,1][i] == 1)
    plt.scatter(index_changes, X_train[:,0][i][index_changes], c='red', label="change point")
    plt.show()'''

'for i in range(100):\n    print(\'picture  \', i)\n    plt.plot(X_train[:,0][i])\n    index_changes = np.where(X_train[:,1][i] == 1)\n    plt.scatter(index_changes, X_train[:,0][i][index_changes], c=\'red\', label="change point")\n    plt.show()'

## 計算Spectral Residual

In [10]:
def spectral_residual(values):
    """
    This method transform a time series into spectral residual series
    :param values: list.
        a list of float values.
    :return: mag: list.
        a list of float values as the spectral residual values
    """
    EPS = 1e-8
    trans = np.fft.fft(values) #傅立葉變換
    mag = np.sqrt(trans.real ** 2 + trans.imag ** 2) #A(f)

    maglog = [np.log(item) if abs(item) > EPS else 0 for item in mag]  #L(f)

    spectral = np.exp(maglog - average_filter(maglog, n=3)) #R(f)

    trans.real = [ireal * ispectral / imag if abs(imag) > EPS else 0
                  for ireal, ispectral, imag in zip(trans.real, spectral, mag)]
    trans.imag = [iimag * ispectral / imag if abs(imag) > EPS else 0
                  for iimag, ispectral, imag in zip(trans.imag, spectral, mag)]

    wave_r = np.fft.ifft(trans)
    mag = np.sqrt(wave_r.real ** 2 + wave_r.imag ** 2) #S(x)

    return mag, maglog

In [11]:
class gen_set():
    def __init__(self, width, train_data):

        self.len = 0
        self.width = width
        self.train_data = train_data
        self.negrawlen = len(self.train_data)
        print('length :', len(self.train_data))
        
        self.len += self.negrawlen
        self.kpineglen = 0
        self.control = 0.

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        
        idx = index % self.negrawlen
        datas = np.array(self.train_data[idx])  # (2,128)
        data = datas[0, :].astype(np.float32)
        lbs = datas[1, :].astype(np.float32)
        
        wave, maglog = spectral_residual(data)
        waveavg = average_filter(wave)
        
        for i in range(self.width):   # 調整 Label的設置 #
            if wave[i] < 0.001 and waveavg[i] < 0.001:
                lbs[i] = 0
                continue
            ratio = wave[i] / waveavg[i]
            if ratio < 1.0 and lbs[i] == 1:
                lbs[i] = 0
            if ratio > 5.0:
                lbs[i] = 1
                
        srscore = abs(wave - waveavg) / (waveavg + 0.01)
        sortid = np.argsort(srscore)
        for idx in sortid[-2:]:
            if srscore[idx] > 5:
                lbs[idx] = 1
                
        resdata = 100 * np.array(wave)
        reslb = lbs
        
        return resdata, reslb, maglog

In [12]:
batch_size = 256
gen_data = gen_set(window, X_train)
data_list = []

for i in range(len(X_train)):
    data_list.append(gen_data.__getitem__(i))
    
X, Y = np.array(data_list)[:,0], np.array(data_list)[:,1]
'''train_x = X[:int(X.shape[0]*0.8)].astype(np.float32)
train_y = Y[:int(Y.shape[0]*0.8)].astype(np.float32)
vali_x = X[int(X.shape[0]*0.8):].astype(np.float32)
vali_y = Y[int(Y.shape[0]*0.8):].astype(np.float32)'''

train_x, vali_x, train_y, vali_y = train_test_split(X.astype(np.float32), Y.astype(np.float32), test_size=0.2, random_state=42)

print('train_x shape', train_x.shape)
print('train_y.shape', train_y.shape)
print('vali_x shape', vali_x.shape)
print('vali_y.shape', vali_y.shape)

train_buf = len(train_x)
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

vali_buf = len(vali_x)
vali_dataset = tf.data.Dataset.from_tensor_slices((vali_x, vali_y))
vali_dataset = vali_dataset.shuffle(buffer_size=vali_buf)
vali_dataset = vali_dataset.batch(batch_size)

length : 8951
train_x shape (7160, 64)
train_y.shape (7160, 64)
vali_x shape (1791, 64)
vali_y.shape (1791, 64)


## SR Result

In [13]:
'''threshold = 3
aa = np.reshape(train_x, [1, -1])
res = np.where(aa > threshold, 1, 0)[0]
bb = np.reshape(train_y, [1, -1])[0]

f1 = f1_score(bb, res)
print('F1', f1)'''

"threshold = 3\naa = np.reshape(train_x, [1, -1])\nres = np.where(aa > threshold, 1, 0)[0]\nbb = np.reshape(train_y, [1, -1])[0]\n\nf1 = f1_score(bb, res)\nprint('F1', f1)"

## log(A(f)) -- 前100筆window

In [14]:
'''for i in range(100):
    print('picture  ', i)
    plt.plot(np.array(data_list)[:,2][i])
    index_changes = np.where(train_y[i] == 1)
    plt.scatter(index_changes, np.array(data_list)[:,2][i][index_changes], c='red', label="change point")
    plt.legend()
    plt.show()'''

'for i in range(100):\n    print(\'picture  \', i)\n    plt.plot(np.array(data_list)[:,2][i])\n    index_changes = np.where(train_y[i] == 1)\n    plt.scatter(index_changes, np.array(data_list)[:,2][i][index_changes], c=\'red\', label="change point")\n    plt.legend()\n    plt.show()'

## 轉SR的Time Series與Label標記情況(前100筆window)

In [15]:
'''for i in range(100):
    print('picture  ', i)
    plt.plot(train_x[i])
    index_changes = np.where(train_y[i] == 1)
    plt.scatter(index_changes, train_x[i][index_changes], c='red', label="change point")
    plt.legend()
    plt.show()'''

'for i in range(100):\n    print(\'picture  \', i)\n    plt.plot(train_x[i])\n    index_changes = np.where(train_y[i] == 1)\n    plt.scatter(index_changes, train_x[i][index_changes], c=\'red\', label="change point")\n    plt.legend()\n    plt.show()'

## 加入CNN做訓練

In [16]:
def make_anomaly_model():
    
    inputs = tf.keras.Input(shape=(window, 1))  

    x = layers.Conv1D(window, kernel_size=1, padding='valid', strides=1, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-4, l2=1e-4))(inputs)
    x = layers.ReLU()(x)
    x = layers.Conv1D(2 * window, kernel_size=1, padding='valid', strides=1, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-4, l2=1e-4))(x)
    x = layers.Flatten()(x)
    x = layers.ReLU()(x)
    x = layers.Dense(4 * window, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-4, l2=1e-4))(x)
    x = layers.ReLU()(x)
    x = layers.Dense(window, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-4, l2=1e-4))(x)  # sigmoid 在 loss 加上 
    model = tf.keras.Model(inputs = inputs, outputs=x)
    
    return model
anomaly_model = make_anomaly_model()
anomaly_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 1)]           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 64, 64)            128       
_________________________________________________________________
re_lu (ReLU)                 (None, 64, 64)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 64, 128)           8320      
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
re_lu_1 (ReLU)               (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               209740

In [17]:
def adjust_lr(epoch, lr):
    cur_lr = lr * (0.5 ** ((epoch + 10) // 10))
    return cur_lr

def calc(pred, true):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for pre, gt in zip(pred, true):
        if gt == 1:
            if pre == 1:
                TP += 1
            else:
                FN += 1
        if gt == 0:
            if pre == 1:
                FP += 1
            else:
                TN += 1
    return TP, FP, TN, FN

In [18]:
class SRCNN():
    
    def __init__(self, anomaly_model, weight, lr):
        self.anomaly_model = anomaly_model
        self.weight = weight
        self.clr = lr
        self.optimizer = tf.keras.optimizers.Adam(lr = self.clr)   #############
 
    def weight_binary_crossentropy(self, y_true, y_pred):
        #weight = 1
        loss = tf.nn.weighted_cross_entropy_with_logits(y_true, y_pred, pos_weight = self.weight)
        #loss = tf.nn.sigmoid_cross_entropy_with_logits(tf.reshape(y_true, [-1,1]), y_pred) 
        return loss

    @tf.function
    def train_model(self, batch_x, batch_y):
        with tf.GradientTape() as tape:
            predict = self.anomaly_model(batch_x, training=True)
            output_loss = self.weight_binary_crossentropy(batch_y, predict)

        grads = tape.gradient(output_loss, anomaly_model.trainable_variables)
        grads, _ = tf.clip_by_global_norm(grads, 5.0)
        self.optimizer.apply_gradients(zip(grads, anomaly_model.trainable_variables))

        return output_loss, predict
    
    def SRCNN_train(self, dataset, show_loss):

        n_epochs = 10000
        start = 0
        pre_loss = 0

        loss_list = []
        now_clr = self.clr
        for epoch in range(n_epochs):
            start = time.time()

            epoch_loss_avg = tf.metrics.Mean()

            for batch_id, (batch) in enumerate(dataset):
                batch_x, batch_y = batch[0], batch[1]

                self.optimizer.lr = self.clr
                loss_, output = self.train_model(tf.expand_dims(batch_x, axis=2), batch_y)
                epoch_loss_avg(loss_)

            loss_list.append(epoch_loss_avg.result())

            if abs(pre_loss - epoch_loss_avg.result()) < 1e-8:
                break
            pre_loss = epoch_loss_avg.result()

            if show_loss:
                epoch_time = time.time() - start
                if epoch % 100 == 0:
                    epoch_time = time.time() - start
                    print('{:4d}: TIME: {:.2f} LOSS: {:.6f} '.format(epoch, epoch_time, epoch_loss_avg.result()))

        final_loss = epoch_loss_avg.result()
        return final_loss, loss_list

## 貝氏最佳化找超參數weight

In [19]:
space  = [Real(10**-3, 10**3, 'uniform', name='weight')]
          #Real(0, 1, 'uniform', name='threshold')]
@use_named_args(space)
def ValidateModel(weight):
    global X,Y
    train_x, vali_x, train_y, vali_y = train_test_split(X.astype(np.float32), Y.astype(np.float32), test_size=0.2, random_state=42)
   
    anomaly_model.load_weights('anomaly_model_weight')
    srcnn = SRCNN(anomaly_model, weight, 1e-6)
    loss, _ = srcnn.SRCNN_train(train_dataset, False)
    
    predicts = tf.reshape(anomaly_model(tf.expand_dims(vali_x, axis=2), training=False), [-1,1])
    vali_y = tf.reshape(vali_y, [-1,1])

    threshold = np.quantile(predicts[vali_y==1], 0.1)  # 0.1的異常被判斷錯誤 的閥值
    pre = [1 if item > threshold else 0 for item in predicts]
    
    '''best_valipre = 0
    for i in range(0, 99, 10):
        threshold = 0.01 + i * 0.01
        pre = [1 if item > threshold else 0 for item in predicts]
        vali_pre = precision_score(vali_y, pre)
        
        if vali_pre > best_valipre:
            best_valipre = vali_pre
            bestthre = threshold'''
        
    #vali_f1 = f1_score(vali_y, pre)
    vali_rec = recall_score(vali_y, pre)

    print('weight :', weight, 'loss :', loss.numpy(), 'threshold', threshold, 'recall score', vali_rec)
    
    return -vali_rec


In [ ]:
search_result = gp_minimize(ValidateModel, space, n_calls = 15)   # n_calls default=100
print("best F1:", -search_result.fun)   # 找出最小的 function value at the minimum.
print(search_result.x)                  # location of the minimum.
weight_opt = search_result.x

weight : 851.6617384285387 loss : 0.14750533 precision score 0.458041958041958
weight : 22.073787135988415 loss : 0.14303635 precision score 0.5101270168211466
weight : 810.5761634155285 loss : 0.3267061 precision score 0.338213293438769
weight : 237.58022798904102 loss : 0.11297866 precision score 0.4818582570362835


## 第二次選

In [23]:
w_2 = weight_opt[0]
space  = [Real(0, w_2 * (1.5), 'uniform', name='weight')]
          #Real(0, 1, 'uniform', name='threshold')]

search_result = gp_minimize(ValidateModel, space, n_calls = 10)   # n_calls default=100
print("best F1:", -search_result.fun)   # 找出最小的 function value at the minimum.
print(search_result.x)                  # location of the minimum.
weight_opt = search_result.x

weight : 1.6481561077844222 loss : 0.060743585 F1 score 0.7013168954889326
weight : 2.597509736638074 loss : 0.052348055 F1 score 0.6660638160444388
weight : 3.1112505912903456 loss : 0.0799901 F1 score 0.696526508226691
weight : 1.6911170793937884 loss : 0.056627713 F1 score 0.6995403259506896
weight : 0.574356291162582 loss : 0.025517024 F1 score 0.6776345042240995
weight : 0.3932635795956472 loss : 0.0237995 F1 score 0.6584004959702418
weight : 3.0782801056852898 loss : 0.07675394 F1 score 0.6971518160438985
weight : 1.8000162301595135 loss : 0.060218327 F1 score 0.7026877251316154
weight : 0.48378331642034855 loss : 0.026766092 F1 score 0.6672774469384639
weight : 0.042881798116385 loss : 0.0044340664 F1 score 0.5669559298962056
best F1: 0.7026877251316154
[1.8000162301595135]


## Train

In [1]:
'''choice_weight = weight_opt[0]
anomaly_model.load_weights('anomaly_model_weight')
srcnn = SRCNN(anomaly_model, choice_weight, 1e-6)
_, loss_list = srcnn.SRCNN_train(train_dataset, True)

plt.plot(loss_list)
plt.show()'''

"choice_weight = weight_opt[0]\nanomaly_model.load_weights('anomaly_model_weight')\nsrcnn = SRCNN(anomaly_model, choice_weight, 1e-6)\n_, loss_list = srcnn.SRCNN_train(train_dataset, True)\n\nplt.plot(loss_list)\nplt.show()"

## Evaluation

In [ ]:
def predict_next(values):
    """
    Predicts the next value by sum up the slope of the last value with previous values.
    Mathematically, g = 1/m * sum_{i=1}^{m} g(x_n, x_{n-i}), x_{n+1} = x_{n-m+1} + g * m,
    where g(x_i,x_j) = (x_i - x_j) / (i - j)
    :param values: list.
        a list of float numbers.
    :return : float.
        the predicted next value.
    """

    if len(values) <= 1:
        raise ValueError(f'data should contain at least 2 numbers')
    
    v_last = values[-1]
    n = len(values)

    slopes = [(v_last - v) / (n - 1 - i) for i, v in enumerate(values[:-1])]

    return values[1] + sum(slopes)

def extend_series(values, extend_num=5, look_ahead=5):
    """
    extend the array data by the predicted next value
    :param values: list.
        a list of float numbers.
    :param extend_num: int, default 5.
        number of values added to the back of data.
    :param look_ahead: int, default 5.
        number of previous values used in prediction.
    :return: list.
        The result array.
    """

    if look_ahead < 1:
        raise ValueError('look_ahead must be at least 1')
    values = list(values)
    extension = [predict_next(values[-look_ahead - 2:-1])] * extend_num
    return values + extension

In [27]:
def get_range_proba(predict, label, delay=7):
    predict = np.array(predict)
    label = np.array(label)

    splits = np.where(label[1:] != label[:-1])[0] + 1
    is_anomaly = label[0] == 1
    new_predict = np.array(predict)
    pos = 0

    for sp in splits:
        if is_anomaly:
            if 1 in predict[pos:min(pos + delay + 1, sp)]:
                new_predict[pos: sp] = 1
            else:
                new_predict[pos: sp] = 0
        is_anomaly = not is_anomaly
        pos = sp
    sp = len(label)

    if is_anomaly:
        if 1 in predict[pos: min(pos + delay + 1, sp)]:
            new_predict[pos: sp] = 1
        else:
            new_predict[pos: sp] = 0

    return new_predict


def reconstruct_label(timestamp, label):
    
    timestamp = np.asarray(timestamp)
    index = np.argsort(timestamp)

    timestamp_sorted = np.asarray(timestamp[index])
    interval = np.min(np.diff(timestamp_sorted))

    label = np.asarray(label, np.int64)
    label = np.asarray(label[index])

    idx = (timestamp_sorted - timestamp_sorted[0]) // interval

    new_label = np.zeros(shape=((timestamp_sorted[-1] - timestamp_sorted[0]) // interval + 1,), dtype=np.int)
    new_label[idx] = label

    return new_label


def reconstruct_series(timestamp, label, predict, delay=7):
    
    label = reconstruct_label(timestamp, label)
    predict = reconstruct_label(timestamp, predict)
    predict = get_range_proba(predict, label, delay)  # 考慮延遲異常點
    return list(label), list(predict)

In [28]:
def evaluate_for_all_series(lst_timestamp_label_predict, delay=7, prt=True):
    labels, predicts = [], []
    for timestamp, label, predict in lst_timestamp_label_predict:
        if timestamp == []:
            continue
        lbl, pdt = reconstruct_series(timestamp, label, predict, delay)
        labels += lbl
        predicts += pdt

    f1 = f1_score(labels, predicts)
    pre = precision_score(labels, predicts)
    rec = recall_score(labels, predicts)
    TP, FP, TN, FN = calc(predicts, labels)
    auc = roc_auc_score(labels, predicts)
    if prt:
        print('precision', pre)
        print('recall', rec)
        print('f1', f1)
        print('TP: {:} FP: {:} TN: {:} FN: {:}'.format(TP, FP, TN, FN))
        print('AUC', auc)
        print('-------------------------------')
    return f1, pre, rec, TP, FP, TN, FN

In [29]:
def window_test(timestamp, value, back_k=0, backaddnum=5, step=32):

    win_size = window
    length = len(timestamp)
    if back_k <= 5:
        back = back_k
    else:
        back = 5
        
    mag_list = []
    for pt in range(int(win_size - backaddnum + back + step), int(length - back), step):
        head = max(0, pt - (win_size - backaddnum))
        tail = min(length, pt)
        wave = np.array(extend_series(value[head:tail + back])) # backaddnum = 增加K個估計點 K=5
        mag, _ = spectral_residual(wave)
        mag_list.append(mag)
    
    return mag_list

In [30]:
def sr_cnn_eval(SR_output, timestamp, value, label, ms_optioin, threshold=0.95, back_k=0, backaddnum=5, step=32):

    def modelwork(x):
        x = 100 * np.array(x)
        x = np.expand_dims(x, axis=2)
        
        #anomaly_model = tf.keras.models.load_model('anomaly_model_yahoo_test3_n.h5', compile=False)
        output = tf.math.sigmoid(anomaly_model(x, training=False))
        aa = np.reshape(output, [1, -1])
        res = np.where(aa > threshold, 1, 0)
        aa = np.reshape(aa, [-1, win_size])
        res = np.reshape(res, [-1, win_size])
        
        return res, aa

    win_size = window
    length = len(timestamp)
    if back_k <= 5:
        back = back_k
    else:
        back = 5
    detres = np.zeros((win_size - backaddnum,))
    scores = np.zeros((win_size - backaddnum,))

    modeloutput, rawout = modelwork(SR_output)
    num = 0
    for pt in range(int(win_size - backaddnum + back + step), int(length - back), step):
        head = max(0, pt - (win_size - backaddnum))
        tail = min(length, pt)
        modeloutput_ = modeloutput[num,:]
        rawout_ = rawout[num,:]
        num += 1
        
        for ipt in range(pt - step - back, pt - back):
            detres = np.append(detres, modeloutput_[ipt - head])
            scores = np.append(scores, rawout_[ipt - head].item())
    
    detres = np.append(detres, np.zeros((length - len(detres),)))   # detres : predict結果 [0,1]顯示
    scores = np.append(scores, np.zeros((length - len(scores),)))   # scores : predict結果 未切割[0,1]顯示
    
    if ms_optioin == 'anomaly':
        last = -1
        
        interval = min([timestamp[i] - timestamp[i - 1] for i in range(1, len(timestamp))]) # 計算兩兩間隔時間 取min
        for i in range(1, len(timestamp)):
            if timestamp[i] - timestamp[i - 1] > interval:
                if last >= 0 and i - last < 1000:
                    detres[i] = 1
                    scores[i] = 1
            if detres[i] == 1:
                last = i
    
    return list(timestamp), label[:], detres[:], scores[:]

In [31]:
def get_score(thres, option):
    total_time = 0
    results = []
    savedscore = []
    
    df_test = pd.read_csv('yahoo_total_data.csv')
    half_len = int(df_test.shape[0]/2)
    in_timestamp, in_value, in_label = df_test['timestamp'][half_len:], df_test['value'][half_len:], df_test['is_anomaly'][half_len:]
    
    
    length = len(in_timestamp)
    
    time_start = time.time()
    SR_output = window_test(np.array(in_timestamp), in_value)
    #print(np.array(SR_output).shape)
    timestamp, label, pre, scores = sr_cnn_eval(SR_output, np.array(in_timestamp), in_value, in_label, option, thres)
    
    time_end = time.time()
    total_time = time_end - time_start
    results.append([timestamp, label, pre])
    savedscore.append([label, scores, timestamp])
    
    return total_time, results, savedscore

In [32]:
thres = 0.5
missing_option = 'anomaly'
delay = 3
window = 64

total_time, results, savedscore = get_score(thres, missing_option)
total_fscore, pre, rec, TP, FP, TN, FN = evaluate_for_all_series(results, delay) # 只拿[0,1]結果
print('time used for making predictions:', total_time, 'seconds')
print('AUC', roc_auc_score(np.array(savedscore)[0][0], np.array(savedscore)[0][1]))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

precision 0.2445902802412203
recall 0.9323867478025692
f1 0.3875228326542082
TP: 1379 FP: 4259 TN: 280745 FN: 100
AUC 0.958721548944442
-------------------------------
time used for making predictions: 206.60539841651917 seconds
AUC 0.9856244559404023


In [33]:
best = 0.
bestthre = 0.
print('delay :', delay)

for i in range(0, 99, 2):  # 考慮不同的 threshold
    newresults = []
    threshold = 0.01 + i * 0.01
    for (flabel, cnnscores, ftimestamp) in savedscore:
        pre = [1 if item > threshold else 0 for item in cnnscores]
        newresults.append([ftimestamp, flabel, pre])
        
    total_fscore, pre, rec, TP, FP, TN, FN = evaluate_for_all_series(newresults, delay, prt=False)
    
    if total_fscore > best:
        best = total_fscore
        best_pre = pre
        best_rec = rec
        bestthre = threshold
        print('threshold', threshold)
        print('F1 best', best)
        print('TP: {:} FP: {:} TN: {:} FN: {:}'.format(TP, FP, TN, FN))
        print()
        
threshold = bestthre
print('best overall threshold :', threshold, 'best F1 score :', best)
print('best precision', best_pre)
print('best recall', best_rec)

delay : 3
threshold 0.01
F1 best 0.03756498406842194
TP: 1456 FP: 74584 TN: 210420 FN: 23

threshold 0.03
F1 best 0.06603212373587151
TP: 1443 FP: 40784 TN: 244220 FN: 36

threshold 0.05
F1 best 0.08793024323083984
TP: 1437 FP: 29769 TN: 255235 FN: 42

threshold 0.06999999999999999
F1 best 0.10734886508352687
TP: 1433 FP: 23786 TN: 261218 FN: 46

threshold 0.09
F1 best 0.125492600052544
TP: 1433 FP: 19926 TN: 265078 FN: 46

threshold 0.11
F1 best 0.14113461158331686
TP: 1428 FP: 17329 TN: 267675 FN: 51

threshold 0.13
F1 best 0.15585980284775464
TP: 1423 FP: 15358 TN: 269646 FN: 56

threshold 0.15000000000000002
F1 best 0.17022809123649457
TP: 1418 FP: 13763 TN: 271241 FN: 61

threshold 0.17
F1 best 0.18485857992030832
TP: 1415 FP: 12415 TN: 272589 FN: 64

threshold 0.19
F1 best 0.19885843140018322
TP: 1411 FP: 11301 TN: 273703 FN: 68

threshold 0.21000000000000002
F1 best 0.2126984126984127
TP: 1407 FP: 10344 TN: 274660 FN: 72

threshold 0.23
F1 best 0.22519634556820003
TP: 1405 FP: 9